In [1]:
MYDIR = "/Users/joregan/Playing/rd_ctm_edit"

In [2]:
from pathlib import Path

In [3]:
WD = Path(MYDIR)

In [4]:
from sync_asr.ctm_edit import ctm_from_file

In [10]:
from sync_asr.riksdag.corrections import _CORRECTIONS

In [11]:
def get_corrections():
    corrections = {}
    for line in _CORRECTIONS.split("\n"):
        if line == "":
            continue
        parts = line.split()
        if not parts[0] in corrections:
            corrections[parts[0]] = []
        corrections[parts[0]].append(parts[1])
    return corrections

In [12]:
corrections = get_corrections()

In [27]:
from string import punctuation

def clean_text(work_ref):
    PUNCT = set(punctuation)
    i = 0
    l = len(work_ref)
    while i < l and work_ref[i] in PUNCT:
        i += 1
    j = -1
    while j >= -l and work_ref[j] in PUNCT:
        j -= 1
    return work_ref[i:j+1]

def compare_text(a, b, lc=False):
    word = clean_text(b)
    if a == word:
        return True
    if lc and a == word.lower():
        return True
    return False


In [17]:
PREFIXES = [
    ("eu", "EU-"),
    ("aendete", "ANDT-"),
    ("rut", "RUT-"),
    ("aendetess", "ANDTS-"),
    ("aett", "A1-"),
    ("adeärr", "ADR-"),
    ("abieff", "ABF-"),
    ("abebé", "ABB-"),
    ("abe", "A/B-"),
    ("hovebe", "HVB-"),
    ("hovebé", "HVB-"),
    ("mu", "MU-"),
    ("ällo", "LO-"),
    ("pisa", "PISA-"),
    ("effen", "FN-"),
    ("jeffen", "FN-"),
    ("äffen", "FN-"),
    ("sab", "SAAB-"),
    ("kex", "KEX-"),
    ("icise", "ICC-"),
]

In [22]:
a = "abebégymnasiet"
b = "ABB-gymnasiet"
p_a = "abebé"
p_b = "ABB-"
a.startswith(p_a) and b.startswith(p_b)
a[len(p_a):] == b[len(p_b):]

True

In [23]:
def checker(a, b):
    if compare_text(a, b, True):
        return True
    word = clean_text(b)
    if a in corrections and word in corrections[a]:
        return True
    if a in corrections and word.lower() in corrections[a]:
        return True
    for pfx in PREFIXES:
        if a.startswith(pfx[0]) and b.startswith(pfx[1]):
            return a[len(pfx[0]):] == b[len(pfx[1]):]
    return False

In [33]:
for file in WD.glob("H201UbU15*"):
    lines = ctm_from_file(file)
    for line in lines:
        line.mark_correct_from_function(checker)

In [34]:
with open("/tmp/foozle", "w") as f:
    for line in lines:
        f.write(str(line) + "\n")